Reservoirs store water between timesteps.  They are nodes in the water network, and receive upstream flows and precipitation, and provide withdrawals to canals.  However, their outflows are determined by an optimization.

# Selecting Reservoirs

The reservoirs that should be modeled as storing water across period, rather than acting like stream nodes, differ depending on the timestep.  The function that provides a list of reservoirs is `getreservoirs(config)`.

In [2]:
include("../src/lib/readconfig.jl")
include("../src/lib/reservoirs.jl")

config = readconfig("../configs/standard.yml")

Dict{Any,Any} with 6 entries:
  "startweather" => 1
  "timestep"     => 6
  "netset"       => "usa"
  "filterstate"  => nothing
  "endmonth"     => "9/2010"
  "startmonth"   => "10/2000"

In [4]:
@doc getreservoirs

Return a DataFrame containing `collection` and `colid` fields matching those in the Water Network.

Any additional columns can be provided, to be used by other components.

Rows may be excluded, to represent that a given reservoir should be modeled as a stream at the specified timestep (in months).


In [3]:
getreservoirs(config)

,collection,colid,area,lat,lon,elev
1,reservoir,1,NA,48.8733,-122.688,NA
2,reservoir,2,145.03944,48.7583,-122.422,NA
3,reservoir,3,556.84785,48.6583,-121.687,NA
4,reservoir,4,556.84785,48.6483,-121.69,NA
5,reservoir,5,2587.40001,48.7317,-121.067,NA
6,reservoir,6,2913.73875,48.7133,-121.13,NA
7,reservoir,7,3001.79841,48.6983,-121.207,NA
8,reservoir,8,769.22703,48.5483,-121.74,NA
9,reservoir,9,NA,48.9317,-119.418,NA
10,reservoir,10,797.71692,48.095,-123.555,NA


To understand the functioning of reservoirs, consider how they work in a very simple three gauge example.